In [2]:
from datasets import load_dataset
from random import randrange

dataset = load_dataset("medalpaca/medical_meadow_medqa", split="train")

print(f"Dataset Size: {len(dataset)}")
print(dataset[randrange(len(dataset))])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset Size: 10178
{'input': 'Q:A 57-year-old man comes to the physician for a follow-up visit. Serum studies show:\nAST 134 U/L\nALT 152 U/L\nHepatitis B surface antigen Positive\nA photomicrograph of the microscopic findings of a liver biopsy is shown. These biopsy findings are most characteristic of which of the following types of inflammatory reactions?"? \n{\'A\': \'Acute inflammation\', \'B\': \'Ischemic necrosis\', \'C\': \'Malignant transformation\', \'D\': \'Granulomatous inflammation\', \'E\': \'Chronic inflammation\'},', 'instruction': 'Please answer with one of the option in the bracket', 'output': 'E: Chronic inflammation'}


In [3]:
def format_prompt(sample):
    return f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{sample["instruction"]}

### Input:
{sample["input"]}

### Response:
{sample["output"]}
"""

In [4]:
from random import randrange

print(format_prompt(dataset[randrange(len(dataset))]))


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please answer with one of the option in the bracket

### Input:
Q:A 27-year-old woman comes to the physician because of a 3-day history of a sore throat and fever. Her temperature is 38.5°C (101.3°F). Examination shows edematous oropharyngeal mucosa and enlarged tonsils with purulent exudate. There is tender cervical lymphadenopathy. If left untreated, which of the following conditions is most likely to occur in this patient?? 
{'A': 'Toxic shock syndrome', 'B': 'Polymyalgia rheumatica', 'C': 'Rheumatoid arthritis', 'D': 'Dilated cardiomyopathy', 'E': 'Erythema multiforme'},

### Response:
D: Dilated cardiomyopathy



In [5]:
from getpass import getpass
hf_token = getpass('Enter your Hugging Face API token:')


Enter your Hugging Face API token:··········


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "meta-llama/Llama-2-7b-chat-hf"
use_flash_attention = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_auth_token=hf_token,
    quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
    torch_dtype=torch.float16
)

model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,use_auth_token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [8]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="finetuned-llama-7b-chat-hf-med",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False
)

In [9]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_prompt,
    args=args,
)

Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
trainer.train()

trainer.save_model()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
)

merged_model = model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged-llama2-7b-chat-hf-med", safe_serialization=True)
tokenizer.save_pretrained("merged-llama2-7b-chat-hf-med")

In [ ]:
from transformers import pipeline
from datasets import load_dataset, load_metric
import numpy as np

pipe = pipeline("text-generation", model="NousResearch/Llama-2-7b-chat-hf", device=0)  # Use device=0 for GPU

dataset = load_dataset("knowrohit07/know_medical_dialogue_v2", split="validation[:1%]")  # Using a small slice for demonstration

rouge = load_metric("rouge")

def generate_summary(dialogue):
    generated_texts = pipe(dialogue, max_length=150, min_length=40, num_return_sequences=1)
    return generated_texts[0]["generated_text"]

results = []
for example in dataset:
    reference = example["summary"]
    generated_summary = generate_summary(example["dialogue"])

    scores = rouge.compute(predictions=[generated_summary], references=[reference])
    results.append(scores)

average_scores = {key: np.mean([score[key].mid.fmeasure for score in results]) for key in results[0]}

print("Average ROUGE Scores:", average_scores)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]